# [RNN으로 BBC기사 분류하기 (자연어 처리)]

In [1]:
# 패키지 수입

import csv
import numpy as np
import nltk     # Natural Language Toolkit
import pandas as pd

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from time import time
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Embedding, SimpleRNN
from keras.layers import Bidirectional # 쌍방향 RNN 기술 : 미래에서 과거 방향 추가
from sklearn.metrics import confusion_matrix, f1_score


In [2]:
# 하이퍼 파라미터
MY_WORDS  = 5000    # 사전의 단어 개수
MY_EMBED  = 64      # 임베딩 차원 / 차원 줄인 후 결과 5000 > 64
MY_HIDDEN = 100     # LSTM의 차원, 현재 단에서 다음 단으로 넘겨주는 데이터의 개수
MY_LEN = 200        # 통일된 기사 길이

MY_SPLIT = 0.8      # TRAIN, TEST 비율
MY_EPOCH = 10       # 반복 학습 수

# 실행 모드 선택
TRAIN_MODE = 1      # 학습 모드인지 (1) 평가 모드인지 선택 (0) / 평가 모드 시, 학습을 종료한 가중치를 파일로 저장

In [3]:
# 제외어 처리 : 흔해 빠진 단어들, 분류에 도움이 안되는 단어들
# 데이터의 부피 감소 = 학습 시간 개선
nltk.download('stopwords')
MY_STOP = set(nltk.corpus.stopwords.words('english'))

print('제외어')
print(MY_STOP)
print('제외어 개수:',len(MY_STOP))

제외어
{'i', 'do', 'yours', 'these', 'above', 've', 'is', 'before', 'don', "that'll", "you've", 'or', 'when', 'myself', 'as', 'over', 'whom', 'to', "hadn't", 'm', 't', 'shan', 'because', 'with', 'his', 'same', 'were', 'where', 'what', 'has', 'up', 'ourselves', 'it', 'such', 'who', "shouldn't", 'how', "won't", "isn't", 'of', 'than', 'didn', "didn't", 'haven', "should've", 'weren', 'o', 'on', 'doesn', 'her', 'more', 'between', 'our', 'no', 'from', 'which', 'then', 'your', 'them', 'just', 'won', 'ours', "couldn't", 'some', 'below', 'he', 'but', 'at', 'out', 'further', 'having', 'while', 'few', 'not', 'during', 'will', 'y', 'ma', 'for', 'itself', 'other', 'this', 'that', 'be', 'under', "you'll", 'off', 'those', 'their', 'isn', 'themselves', 'hasn', 'down', 'yourselves', "weren't", 's', 'does', "hasn't", "haven't", 'again', 'the', 'are', "she's", 'mightn', 'against', 'can', 'wasn', 'both', 'was', 'they', 'am', 'all', 'hers', 'a', 'll', 'now', 'so', 'herself', 'into', 'him', 'himself', 'needn',

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wds66\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# 데이터 저장 창고
original = []   # 기사 원본
finished = []   # 기사 처리본
labels = []     # 기사 카테고리

# 데이터 읽기
path = 'C:/Users/wds66/Desktop/work/2022_Georgia/bbc-text.csv'

with open(path,'r') as file:
    # 컬럼 헤딩 처리
    finger = csv.reader(file)
    header = next(finger)
    # print(header)
    
    # 기사 하나씩 처리
    for row in finger:
        # print(row)
        labels.append(row[0])
        original.append(row[1])
        news = row[1]
        
        # 제외어 처리
        for word in MY_STOP:
            mask = ' ' + word + ' '
            news = news.replace(mask,' ')
        # print(news)
        finished.append(news)

print('처리한 기사 수:',len(finished))

처리한 기사 수: 2225


In [5]:
# 샘플 출력
print('샘플 기사 원본:',original[123])
print('글자 수 :',len(original[123]))
print('단어 수 :',len(original[123].split()))

print('샘플 기사 제외어 처리본 :',finished[123])
print('제외어 처리 하고 단어 수:',len(finished[123].split()))

샘플 기사 원본: screensaver tackles spam websites net users are getting the chance to fight back against spam websites  internet portal lycos has made a screensaver that endlessly requests data from sites that sell the goods and services mentioned in spam e-mail. lycos hopes it will make the monthly bandwidth bills of spammers soar by keeping their servers running flat out. the net firm estimates that if enough people sign up and download the tool  spammers could end up paying to send out terabytes of data.   we ve never really solved the big problem of spam which is that its so damn cheap and easy to do   said malte pollmann  spokesman for lycos europe.  in the past we have built up the spam filtering systems for our users   he said   but now we are going to go one step further.    we ve found a way to make it much higher cost for spammers by putting a load on their servers.  by getting thousands of people to download and use the screensaver  lycos hopes to get spamming websites constantly 

In [6]:
# 기사 데이터 토큰 처리
# 단어를 고유의 정수로 전환
# OOV: out-of-vocabulary
A_token = Tokenizer(num_words=MY_WORDS,
                    oov_token='!')
A_token.fit_on_texts(finished)

# 단어 토큰화 (num_words 미적용)
# print(A_token.word_counts)
print('전체 기사에 등장한 고유한 단어 수 :',len(A_token.word_counts))
# print('토큰화 결과 :',A_token.word_index)

# 기사 토큰화 (num_words 적용)
finished = A_token.texts_to_sequences(finished)
print(type(finished))

전체 기사에 등장한 고유한 단어 수 : 29698
<class 'list'>


In [7]:
# 토큰화 기사 샘플
# print('원본 :',original[123])
# print('토큰화 처리본:',finished[123])

# 기사 길이 통계
for i in range(10):
    print('기사:',i,', 길이:',len(finished[i]))    
longest = max([len(x) for x in finished])
print('제일 긴 기사의 길이:',longest)
shortest = min([len(x) for x in finished])
print('제일 짧은 기사의 길이:',shortest)


기사: 0 , 길이: 425
기사: 1 , 길이: 192
기사: 2 , 길이: 132
기사: 3 , 길이: 275
기사: 4 , 길이: 188
기사: 5 , 길이: 355
기사: 6 , 길이: 153
기사: 7 , 길이: 113
기사: 8 , 길이: 102
기사: 9 , 길이: 136
제일 긴 기사의 길이: 2279
제일 짧은 기사의 길이: 50


In [8]:
# 기사 길이 통일
finished = pad_sequences(finished,
                            truncating='pre',
                            padding = 'pre',
                            maxlen=MY_LEN)

# 기사 길이 통계
for i in range(10):
    print('기사:',i,', 길이:',len(finished[i]))    
longest = max([len(x) for x in finished])
print('제일 긴 기사의 길이:',longest)
shortest = min([len(x) for x in finished])
print('제일 짧은 기사의 길이:',shortest)

기사: 0 , 길이: 200
기사: 1 , 길이: 200
기사: 2 , 길이: 200
기사: 3 , 길이: 200
기사: 4 , 길이: 200
기사: 5 , 길이: 200
기사: 6 , 길이: 200
기사: 7 , 길이: 200
기사: 8 , 길이: 200
기사: 9 , 길이: 200
제일 긴 기사의 길이: 200
제일 짧은 기사의 길이: 200


In [9]:
# 라벨 토큰화
C_token = Tokenizer()
C_token.fit_on_texts(labels)

# 단어를 토큰화
print('라벨 단어의 빈도수:',C_token.word_counts)
print('토큰화 결과',C_token.word_index)

# 전체 기사 카테고리 토큰화
labels = C_token.texts_to_sequences(labels)
# print(labels)

라벨 단어의 빈도수: OrderedDict([('tech', 401), ('business', 510), ('sport', 511), ('entertainment', 386), ('politics', 417)])
토큰화 결과 {'sport': 1, 'business': 2, 'politics': 3, 'tech': 4, 'entertainment': 5}


In [10]:
# 데이터 4분할
print(type(finished))
print(type(labels))
labels = np.array(labels)

X_train, X_test, Y_train, Y_test = train_test_split(finished,
                                                    labels,
                                                    train_size=MY_SPLIT,
                                                    shuffle=False)

print('X_train 모양 :',X_train.shape)
print('Y_train 모양 :',Y_train .shape)
print('X_test 모양  :',X_test.shape)
print('Y_test 모양  :',Y_test.shape)

<class 'numpy.ndarray'>
<class 'list'>
X_train 모양 : (1780, 200)
Y_train 모양 : (1780, 1)
X_test 모양  : (445, 200)
Y_test 모양  : (445, 1)


In [11]:
# 기사 분류기 구현
model = Sequential()

# 임베딩 층 추가
model.add(Embedding(input_dim=MY_WORDS,
                    output_dim=MY_EMBED,
                    input_length=MY_LEN))

# LSTM 추가
model.add(LSTM(units=MY_HIDDEN,
                input_shape=(MY_LEN,MY_EMBED)))

# 출력층
model.add(Dense(units=6,
            activation='softmax'))

# RNN 요약
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 64)           320000    
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense (Dense)               (None, 6)                 606       
                                                                 
Total params: 386,606
Trainable params: 386,606
Non-trainable params: 0
_________________________________________________________________


In [12]:
# 13번 셀

# RNN 학습
model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['acc'])
# crossentropy                      >> 2진 분류 / 원-핫 ㅇㅇ
# categorical_crossentropy          >> 3이상    / 원-핫 ㅇㅇ
# sparse_categorical_crossentropy   >> 3이상    / 원-핫 ㄴㄴ 근데 crossentropy는 쓰고 싶어

print('학습 시작')
begin = time()

model.fit(X_train,Y_train,
            epochs=MY_EPOCH,
            verbose=1)

end = time()
print('총 학습 시간:',end-begin)

학습 시작
Epoch 1/10
56/56 [==============================] - 7s 96ms/step - loss: 1.6481 - acc: 0.2489
Epoch 2/10
56/56 [==============================] - 6s 100ms/step - loss: 1.3239 - acc: 0.4404
Epoch 3/10
56/56 [==============================] - 6s 102ms/step - loss: 0.9011 - acc: 0.7101 1s - loss: 
Epoch 4/10
56/56 [==============================] - 6s 104ms/step - loss: 0.4700 - acc: 0.8876
Epoch 5/10
56/56 [==============================] - 6s 105ms/step - loss: 0.1578 - acc: 0.9663
Epoch 6/10
56/56 [==============================] - 6s 106ms/step - loss: 0.0843 - acc: 0.9775
Epoch 7/10
56/56 [==============================] - 6s 105ms/step - loss: 0.0301 - acc: 0.9938
Epoch 8/10
56/56 [==============================] - 6s 107ms/step - loss: 0.0235 - acc: 0.9955
Epoch 9/10
56/56 [==============================] - 6s 112ms/step - loss: 0.0256 - acc: 0.9966
Epoch 10/10
56/56 [==============================] - 6s 113ms/step - loss: 0.0204 - acc: 0.9972
총 학습 시간: 59.99891138076782


In [13]:
# RNN 평가
score = model.evaluate(X_test,Y_test)
print('정확도 :',score[1])

14/14 [==============================] - 1s 35ms/step - loss: 0.1896 - acc: 0.9438
정확도 : 0.9438202381134033


In [17]:
# RNN 예측
# 토큰화 결과 :{1 >> 스포츠 / 2 >> 비즈니스 / 3 >> 정치 / 4 >> 기술 / 5 >> 엔터}
pred = model.predict(X_test)
print('평가용 0번 문제 예측 :',pred[0].argmax())
print('평가용 0번 문제 정답 :',Y_test[0])

평가용 0번 문제 예측 : 5
평가용 0번 문제 정답 : [5]


In [34]:
# BBC 기사로 실습
news = ['US President Joe Biden has called for a historic change to Senate rules as he seeks to overhaul the countrys election laws.\
In an impassioned speech, he said he supported changes that would allow his voting reforms to be passed without the support of opposition Republicans.\
Misgivings from two senators in his party are hampering his plans, and no Republicans have backed them.\
Currently, a majority of 60% is needed to pass most legislation in the Senate.\
And with the upper chamber of Congress split 50-50 between the two parties, Mr Bidens sweeping election bills are almost certain not to pass unless there is a change to that rule.\
Such a change is unlikely, analysts say, as it would require the support of every Democrat in the Senate as well as the tie-breaking vote of the vice-president.']
# print(news)

# 토큰화 처리
print(news)
news = A_token.texts_to_sequences(news)
print('토큰화 결과:',news)
print(len(news[0]))

# 길이 처리
news = pad_sequences(news,
                        truncating='pre',
                        padding = 'pre',
                        maxlen=MY_LEN)
print('단어 수:',len(news[0]))

# 예측 결과
pred = model.predict(news)
print('예측 결과:',pred[0].argmax())

['US President Joe Biden has called for a historic change to Senate rules as he seeks to overhaul the countrys election laws.In an impassioned speech, he said he supported changes that would allow his voting reforms to be passed without the support of opposition Republicans.Misgivings from two senators in his party are hampering his plans, and no Republicans have backed them.Currently, a majority of 60% is needed to pass most legislation in the Senate.And with the upper chamber of Congress split 50-50 between the two parties, Mr Bidens sweeping election bills are almost certain not to pass unless there is a change to that rule.Such a change is unlikely, analysts say, as it would require the support of every Democrat in the Senate as well as the tie-breaking vote of the vice-president.']
토큰화 결과: [[9, 194, 3436, 1, 1, 152, 1662, 1347, 4023, 312, 588, 1, 506, 1, 1, 3537, 588, 1, 1071, 1, 49, 781, 602, 1, 1, 746, 1, 2, 1, 3076, 618, 755, 4, 441, 1, 1516, 1741, 588, 2398, 1684, 185, 1071, 2